### import libraries

In [1]:
using QuantumOptics, PyPlot, PyCall, JLD

### define parameters for the position and momentum basis

In [2]:
#coupling factor
g=-29.45

#position and momentum basis in x
x_min= -20
x_max = 20
x_steps = 62
dx = (x_max - x_min) / x_steps
b_x = PositionBasis(x_min, x_max, x_steps)
b_px = MomentumBasis(b_x)
xsample = samplepoints(b_x)

#position and momentum basis in y
y_min= -10
y_max = 10
y_steps = 32
dy = (y_max - y_min) / y_steps
b_y = PositionBasis(y_min, y_max, y_steps)
b_py = MomentumBasis(b_y)
ysample = samplepoints(b_y)

#momentum:
px = momentum(b_px)
py = momentum(b_py)

#TransformationOperator for composite bases:
b_comp_x = tensor(b_x, b_y)
b_comp_p = tensor(b_px, b_py)
Txp = transform(b_comp_x, b_comp_p)
Tpx = transform(b_comp_p, b_comp_x)

#Hamiltonians:
Hkinx = LazyTensor(b_comp_p, [1, 2], [px^2/2, one(b_py)])
Hkiny = LazyTensor(b_comp_p, [1, 2], [one(b_px), py^2/2])

Hkinx_FFT = LazyProduct(Txp, Hkinx, Tpx)
Hkiny_FFT = LazyProduct(Txp, Hkiny, Tpx)

Hpsi = diagonaloperator(b_comp_x, Ket(b_comp_x).data) # proportional zu |psi|^2
H0 = LazySum(Hkinx_FFT, Hkiny_FFT, Hpsi)

LazySum(dim=1984x1984)
  basis: [Position(xmin=-20.0, xmax=20.0, N=62) ⊗ Position(xmin=-10.0, xmax=10.0, N=32)]
  operators: 3

### parameters for two gaussian wavepacket

In [3]:
#position
x0 = 5
y0 = 0

#momentum
p0_x = 0
p0_y = 0

#size
sigmax = 2
sigmay= 2

#first wavepacket
psix1 = gaussianstate(b_x, -x0, p0_x, sigmax)
psiy1 = gaussianstate(b_y, y0, p0_y, sigmay)
psi1 = psix1 ⊗ psiy1

#second wavepacket
psix2 = gaussianstate(b_x, x0, -p0_x, sigmax)
psiy2 = gaussianstate(b_y, y0, p0_y, sigmay)
psi2 = psix2 ⊗ psiy2

psi = normalize(psi1 + psi2)

;

In [4]:
function H(t, psi) # updating state-dependent term in Hamiltonian H
	Hpsi.data.nzval .= g*abs2.(psi.data)
	return H0
end

H (generic function with 1 method)

## calculating evolution of the wavepackets - this is where the magic happens!

In [5]:
T = [0:1:100;]
tout, psit = timeevolution.schroedinger_dynamic(T, psi, H) 
density = [reshape(abs2.(psi.data), (x_steps, y_steps))' for psi=psit]

;

### plotting the results

In [6]:
#function for the subplots - because pyplot in julia takes no loops
function make_plot(idx, textx_position, texty_position)
    c = contourf(xsample, ysample, density[idx], clev )
    text(textx_position, texty_position, L"time $t$:"*"$(T[idx])", color="white") #Zeitstempel
    text(textx_position, texty_position+2, L"norm $=$ "*"$(round(norm(psit[idx]),digits = 7))", color = "white") # Normstempel
    xlabel(L"$x / \bar{x}$")
    ylabel(L"$y / \bar{x}$")
    return c
end

make_plot (generic function with 1 method)

In [8]:
#Daten für Plot:
maxval = maximum(density[10])
textx_position = minimum(xsample)+1
texty_position = minimum(ysample)+1
rc("font", family="serif") #Schrift für Plot
clev = LinRange(0,maxval, 10)
fig = figure(figsize=(10,8)) 

i=1
subplot(221,aspect="equal")
make_plot(i, textx_position, texty_position)

i=8
subplot(222,aspect="equal")
make_plot(i, textx_position, texty_position)

i=32
subplot(223,aspect="equal")
make_plot(i, textx_position, texty_position)

i=42
subplot(224,aspect="equal")
make_plot(i, textx_position, texty_position)

#Colorbar:
cbar_ax = fig.add_axes([1, 0.145, 0.015, 0.7])
cbar = fig.colorbar(c, cax=cbar_ax, ticks=[0, maxval])
cbar.set_label("Probability density")
cbar.set_ticklabels(["Low", "High"])

tight_layout()
show()

UndefVarError: [91mUndefVarError: c not defined[39m